# Plugin to model temporal dynamics in microbial time-series

![alt text](overview.png)

## Import

In [ ]:
import os
import warnings

import pandas as pd
import qiime2 as q2

from q2_time.compare_models import compare_config, eval_simulations
from q2_time.engineer_features import transform_features
from q2_time.model import fit_n_predict_model, run_models

# from q2_time.simulate_data import simulate_data

# todo: remove warning suppression after demo
warnings.filterwarnings("ignore")

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Real dataset (5 out of 8)

In [ ]:
# # simulated data
# ft, md = simulate_data(100)

In [ ]:
path2data = "/home/adamova/cloud/adamova/infant_data/220728_monthly/"

In [ ]:
path2md = os.path.join(path2data, "metadata_proc_v20230110.tsv")
md = pd.read_csv(path2md, sep="\t", index_col=0)
md = md[["host_id", "age_days"]].copy()
print(md.shape)
md.head()

In [ ]:
path2ft = os.path.join(path2data, "all_otu_table.qza")
ft = q2.Artifact.load(path2ft).view(pd.DataFrame)
print(ft.shape)
ft.head()

## Transform features

In [ ]:
ft_t_c = transform_features(ft, "clr")
ft_t_i = transform_features(ft, "ilr")
ft_t_a = transform_features(ft, "alr", 5)

## Train & compare models

### Compare settings: different regressor on raw data

In [ ]:
## Static model
lin_reg, _, _ = fit_n_predict_model(md, ft, "age_days", "host_id", "LinRegressor")

rf_reg, _, _ = fit_n_predict_model(md, ft, "age_days", "host_id", "RFRegressor")

In [ ]:
compare_config({"linreg": lin_reg, "rf": rf_reg})

### Compare performance: different transforms & regressors

In [ ]:
# define simulations to run:
simulations = {
    "raw": [md, ft],
    "clr": [md, ft_t_c],
    "alr": [md, ft_t_a],
    "ilr": [md, ft_t_i],
}
simulations_all = {}
for key, item in simulations.items():
    simulations_all[key + "_LinReg"] = item + ["LinRegressor"]
    simulations_all[key + "_RFReg"] = item + ["RFRegressor"]

print(simulations_all.keys())

In [ ]:
sim_results = run_models(simulations_all, 10)

In [ ]:
eval_simulations(sim_results, set="test")

In [ ]:
eval_simulations(sim_results, set="train")